# "Berlin Net"

In [1]:
from comet_ml import Experiment
import keras
from keras import models, layers
import numpy as np
import matplotlib.pyplot as plt
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
import sys
sys.path.append('../')
from utils import DataFeed
from keras import regularizers

Using TensorFlow backend.


## Model

Similar Architecture to [paper](https://github.com/twerkmeister/iLID/blob/master/Deep%20Audio%20Paper%20Thomas%20Werkmeister%2C%20Tom%20Herold.pdf)

Changes:
- add dropout
- no pooling stride
- remove batch normalization

In [2]:
model = models.Sequential()
model.add(Melspectrogram(n_dft=512, input_shape=(1, 5 * 16000,),
                         padding='same', sr=16000, n_mels=28,
                         fmin=0.0, fmax=10000, power_melgram=1.0,
                         return_decibel_melgram=True, trainable_fb=False,
                         trainable_kernel=False))
model.add(Normalization2D(str_axis='data_sample'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
melspectrogram_1 (Melspectro (None, 28, 313, 1)        270364    
_________________________________________________________________
normalization2d_1 (Normaliza (None, 28, 313, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 311, 64)       640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 155, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 153, 64)       36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 76, 64)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 74, 128)        73856     
__________

In [3]:
model.compile(optimizer='RMSprop',
              metrics=['accuracy', 'mae'],
              loss='categorical_crossentropy')

## Train on 50'000 samples

In [3]:
data_path = '../preprocessing/preprocessed_data'

In [3]:
train_data, train_labels = DataFeed.Dataset.create(data_path, ['train/voxforge', 'train/youtube'], num=50000, use_premade=False)

In [5]:
val_data, val_labels = DataFeed.Dataset.create(data_path, ['val/youtube', 'val/voxforge'], num=-1, shuffle=True)

In [27]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=5),
             keras.callbacks.ModelCheckpoint('berlin_net_small.h5', monitor='val_loss', save_best_only=True)]

In [37]:
history = model.fit(x=train_data,
                    y=train_labels,
                    batch_size=128, 
                    epochs=16,
                    validation_data=(val_data, val_labels), 
                    shuffle=True,
                    callbacks=callbacks)

Train on 50000 samples, validate on 10000 samples
Epoch 1/16
50000/50000 [==============================] - 44s 886us/step - loss: 0.9326 - acc: 0.5491 - mean_absolute_error: 0.3641 - val_loss: 0.8406 - val_acc: 0.5901 - val_mean_absolute_error: 0.3074
Epoch 2/16
50000/50000 [==============================] - 41s 810us/step - loss: 0.5924 - acc: 0.7528 - mean_absolute_error: 0.2236 - val_loss: 0.4287 - val_acc: 0.8398 - val_mean_absolute_error: 0.1695
Epoch 3/16
50000/50000 [==============================] - 41s 810us/step - loss: 0.4273 - acc: 0.8314 - mean_absolute_error: 0.1551 - val_loss: 0.3144 - val_acc: 0.8838 - val_mean_absolute_error: 0.1360
Epoch 4/16
50000/50000 [==============================] - 40s 807us/step - loss: 0.3397 - acc: 0.8687 - mean_absolute_error: 0.1229 - val_loss: 0.4453 - val_acc: 0.8161 - val_mean_absolute_error: 0.1481
Epoch 5/16
50000/50000 [==============================] - 40s 808us/step - loss: 0.2879 - acc: 0.8892 - mean_absolute_error: 0.1028 - val_

In [38]:
history = model.fit(x=train_data,
                    y=train_labels,
                    batch_size=128, 
                    epochs=5,
                    validation_data=(val_data, val_labels), 
                    shuffle=True,
                    callbacks=callbacks)

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 41s 812us/step - loss: 0.1393 - acc: 0.9522 - mean_absolute_error: 0.0453 - val_loss: 0.3222 - val_acc: 0.9086 - val_mean_absolute_error: 0.0709
Epoch 2/5
50000/50000 [==============================] - 41s 816us/step - loss: 0.1340 - acc: 0.9545 - mean_absolute_error: 0.0430 - val_loss: 0.4171 - val_acc: 0.8355 - val_mean_absolute_error: 0.1259
Epoch 3/5
11008/50000 [=====>........................] - ETA: 27s - loss: 0.1268 - acc: 0.9555 - mean_absolute_error: 0.0424

KeyboardInterrupt: 

In [39]:
small_model = keras.models.load_model('berlin_net_small.h5', custom_objects={'Melspectrogram':Melspectrogram, 
                                                                             'Normalization2D': Normalization2D})
tet_data, test_labels = DataFeed.Dataset.create(data_path, ['test/youtube', 'test/voxforge'], num=-1, shuffle=True)
small_model.evaluate(x=test_data,
                     y=test_labels), model.metrics_names

int_axis=0 passed but is ignored, str_axis is used instead.
10000/10000 [==============================] - 6s 568us/step


([0.18541387659870087, 0.9369, 0.05716399599462747],
 ['loss', 'acc', 'mean_absolute_error'])

## Train Model on 100'000 samples

In [6]:
training_generator = DataFeed.DataGenerator(data_path, ['train/voxforge', 'train/youtube'], num=-1, batch_size=128)

In [7]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3),
             keras.callbacks.ModelCheckpoint('berlin_net.h5', monitor='val_loss', save_best_only=True)]

In [8]:
history = model.fit_generator(generator=training_generator,
                              epochs=10,
                              validation_data=(val_data, val_labels), 
                              shuffle=True,
                              use_multiprocessing=True,
                              workers=8,
                              max_queue_size=20,          
                              callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 113s 181ms/step - loss: 0.8290 - acc: 0.6133 - mean_absolute_error: 0.3218 - val_loss: 0.5446 - val_acc: 0.7686 - val_mean_absolute_error: 0.2212
Epoch 2/10
625/625 [==============================] - 107s 171ms/step - loss: 0.4618 - acc: 0.8158 - mean_absolute_error: 0.1703 - val_loss: 0.3889 - val_acc: 0.8408 - val_mean_absolute_error: 0.1507
Epoch 3/10
625/625 [==============================] - 103s 165ms/step - loss: 0.3340 - acc: 0.8724 - mean_absolute_error: 0.1192 - val_loss: 0.4751 - val_acc: 0.7852 - val_mean_absolute_error: 0.1743
Epoch 4/10
625/625 [==============================] - 106s 170ms/step - loss: 0.2779 - acc: 0.8951 - mean_absolute_error: 0.0981 - val_loss: 0.2521 - val_acc: 0.9102 - val_mean_absolute_error: 0.0930
Epoch 5/10
625/625 [==============================] - 104s 167ms/step - loss: 0.2414 - acc: 0.9103 - mean_absolute_error: 0.0849 - val_loss: 0.4489 - val_acc: 0.8409 - val_mean_absolute_error: 0.1257


In [9]:
history = model.fit_generator(generator=training_generator,
                              epochs=5,
                              validation_data=(val_data, val_labels), 
                              shuffle=True,
                              use_multiprocessing=True,
                              workers=8,
                              max_queue_size=20,          
                              callbacks=callbacks)

Epoch 1/5
625/625 [==============================] - 105s 168ms/step - loss: 0.1652 - acc: 0.9411 - mean_absolute_error: 0.0558 - val_loss: 0.1496 - val_acc: 0.9478 - val_mean_absolute_error: 0.0489
Epoch 2/5
625/625 [==============================] - 108s 172ms/step - loss: 0.1644 - acc: 0.9411 - mean_absolute_error: 0.0552 - val_loss: 0.1484 - val_acc: 0.9487 - val_mean_absolute_error: 0.0492
Epoch 3/5
625/625 [==============================] - 104s 166ms/step - loss: 0.1600 - acc: 0.9442 - mean_absolute_error: 0.0530 - val_loss: 0.1548 - val_acc: 0.9429 - val_mean_absolute_error: 0.0606
Epoch 4/5
625/625 [==============================] - 107s 171ms/step - loss: 0.1552 - acc: 0.9455 - mean_absolute_error: 0.0518 - val_loss: 0.1513 - val_acc: 0.9458 - val_mean_absolute_error: 0.0539
Epoch 5/5
625/625 [==============================] - 105s 168ms/step - loss: 0.1536 - acc: 0.9464 - mean_absolute_error: 0.0502 - val_loss: 0.1763 - val_acc: 0.9410 - val_mean_absolute_error: 0.0514


In [5]:
big_model = keras.models.load_model('berlin_net/berlin_net.h5', custom_objects={'Melspectrogram':Melspectrogram, 
                                                                             'Normalization2D': Normalization2D})
#test_data, test_labels = DataFeed.Dataset.create(data_path, ['test/youtube', 'test/voxforge'], num=-1, shuffle=True)
big_model.evaluate(x=test_data,
                   y=test_labels), big_model.metrics_names

int_axis=0 passed but is ignored, str_axis is used instead.
10000/10000 [==============================] - 5s 547us/step


([0.141640976549685, 0.9511, 0.04648835944905877],
 ['loss', 'acc', 'mean_absolute_error'])